In [1]:
from pathlib import Path
import pandas as pd


# Mode (test vs raw) + paths

MODE = "test"   # change to "raw" for full dataset

PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
RESULTS_DIR = PROJECT_ROOT / "results" / MODE
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

print("Mode:", MODE)
print("Project root:", PROJECT_ROOT)
print("Results dir:", RESULTS_DIR.resolve())

Mode: test
Project root: /Users/matysprecloux/Desktop/Master IEAP/Code MOTTET/Defense /SYNCOGESTM2
Results dir: /Users/matysprecloux/Desktop/Master IEAP/Code MOTTET/Defense /SYNCOGESTM2/results/test


In [2]:
qdm_path = RESULTS_DIR / "vicon_QDM_wrists_head_mm.xlsx"
sw_path  = RESULTS_DIR / "vicon_shoulder_width_by_csv.xlsx"

qdm = pd.read_excel(qdm_path)
sw  = pd.read_excel(sw_path)

print("QDM rows:", len(qdm), "| columns:", qdm.shape[1])
print("SW rows:", len(sw),  "| columns:", sw.shape[1])
qdm.head()

QDM rows: 3 | columns: 22
SW rows: 3 | columns: 8


,csv,file,P1_poignet_D_QDM_mm,P1_poignet_D_nsteps,P1_poignet_G_QDM_mm,P1_poignet_G_nsteps,P1_QDM_WRISTS_mm,P1_Tempe_D_QDM_mm,P1_Tempe_D_nsteps,P1_Tempe_G_QDM_mm,...,P2_2poignet_D_QDM_mm,P2_2poignet_D_nsteps,P2_2poignet_G_QDM_mm,P2_2poignet_G_nsteps,P2_QDM_WRISTS_mm,P2_2Tempe_D_QDM_mm,P2_2Tempe_D_nsteps,P2_2Temps_G_QDM_mm,P2_2Temps_G_nsteps,P2_QDM_HEAD_mm
0,/Users/matysprecloux/Desktop/Master IEAP/Code ...,SEATEDD01.csv,11257.605164,18011,5086.439843,18011,16344.045006,15748.775325,18011,18437.193954,...,6990.478950,18011,4791.810419,18011,11782.289369,10070.014078,18011,10379.521199,18011,20449.535277
1,/Users/matysprecloux/Desktop/Master IEAP/Code ...,SEMID01.csv,8668.166077,17999,6035.964257,17999,14704.130334,13942.428120,17999,14935.841734,...,7473.133557,17999,8656.559835,17999,16129.693392,8852.532002,17999,9113.597368,17999,17966.129370
2,/Users/matysprecloux/Desktop/Master IEAP/Code ...,STANDINGD01.csv,21176.211919,17999,18263.692630,17999,39439.904549,15582.855481,17999,17012.960766,...,23389.330425,17999,22248.213123,17999,45637.543548,13910.023848,17999,13998.875022,17999,27908.898870


In [3]:
# QDM: column 'file' like "SEATEDD01.csv"
qdm["video_id"] = qdm["file"].astype(str).str.replace(".csv", "", regex=False)

# Shoulder-width: column 'file' also exists in our pipeline, but we keep your robust approach:
# if "csv" contains the full path, use stem (filename without extension)
if "csv" in sw.columns:
    sw["video_id"] = sw["csv"].astype(str).apply(lambda x: Path(x).stem)
else:
    # fallback if sw already has "file" like "SEATEDD01.csv"
    sw["video_id"] = sw["file"].astype(str).str.replace(".csv", "", regex=False)

qdm[["file", "video_id"]].head()

,file,video_id
0,SEATEDD01.csv,SEATEDD01
1,SEMID01.csv,SEMID01
2,STANDINGD01.csv,STANDINGD01


In [4]:
out = qdm.merge(
    sw[[
        "video_id",
        "P1_shoulder_width_median_mm",
        "P2_shoulder_width_median_mm"
    ]],
    on="video_id",
    how="left"
)

missing = out["P1_shoulder_width_median_mm"].isna().sum()
print(f"Rows without shoulder width: {missing}/{len(out)}")

if missing > 0:
    display(out.loc[out["P1_shoulder_width_median_mm"].isna(), ["video_id"]].head(20))

Rows without shoulder width: 0/3


In [5]:
# Wrists
out["P1_QDM_WRISTS_norm"] = out["P1_QDM_WRISTS_mm"] / out["P1_shoulder_width_median_mm"]
out["P2_QDM_WRISTS_norm"] = out["P2_QDM_WRISTS_mm"] / out["P2_shoulder_width_median_mm"]

# Head
out["P1_QDM_HEAD_norm"] = out["P1_QDM_HEAD_mm"] / out["P1_shoulder_width_median_mm"]
out["P2_QDM_HEAD_norm"] = out["P2_QDM_HEAD_mm"] / out["P2_shoulder_width_median_mm"]

out[[
    "video_id",
    "P1_QDM_WRISTS_mm", "P1_shoulder_width_median_mm", "P1_QDM_WRISTS_norm",
    "P2_QDM_WRISTS_mm", "P2_shoulder_width_median_mm", "P2_QDM_WRISTS_norm"
]].head()

,video_id,P1_QDM_WRISTS_mm,P1_shoulder_width_median_mm,P1_QDM_WRISTS_norm,P2_QDM_WRISTS_mm,P2_shoulder_width_median_mm,P2_QDM_WRISTS_norm
0,SEATEDD01,16344.045006,281.600411,58.039848,11782.289369,305.506314,38.566435
1,SEMID01,14704.130334,284.547768,51.675437,16129.693392,336.097952,47.991049
2,STANDINGD01,39439.904549,290.321487,135.849072,45637.543548,345.850784,131.957323


In [6]:
out_path = RESULTS_DIR / "vicon_QDM_wrists_head_normByShoulders.xlsx"
out.to_excel(out_path, index=False)

print("✅ Saved:", out_path.resolve())

✅ Saved: /Users/matysprecloux/Desktop/Master IEAP/Code MOTTET/Defense /SYNCOGESTM2/results/test/vicon_QDM_wrists_head_normByShoulders.xlsx
